In [9]:
# Install PySpark
try:
    import pyspark
except ImportError:
    print("pyspark not found. Installing...")
    !pip install pyspark > pyspark.log.txt
    print("pyspark installed successfully!")

In [17]:
# Import necessary libraries
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from matplotlib import cm
import numpy as np 
import pandas as pd
import seaborn as sns
import warnings
import timeit

# package imports
from pyspark.sql.functions import col, isnull, broadcast, udf, count, when, isnan, lpad, to_timestamp, concat, to_utc_timestamp, lit 
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import Bucketizer

from pyspark.sql import SQLContext
from pyspark.sql.types import DoubleType
from functools import reduce

import numpy as np
import math

import networkx as nx
import matplotlib.pyplot as plt
import geopandas

# package imports
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from pyspark.sql.types import DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, OneHotEncoder , VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import PCA
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, isnull, broadcast, udf, count, when, isnan, lpad, to_timestamp, concat, to_utc_timestamp, expr, unix_timestamp, avg, round, lag, to_date, row_number, explode, array, lit, hour
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from pyspark.sql.types import DoubleType

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_squared_error

import seaborn as sns
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# Suppress all warnings
warnings.filterwarnings("ignore")

In [11]:
# Create a Spark session
spark = SparkSession.builder.appName("Spark").getOrCreate()

In [14]:
# Load airlines pre-processed data. 
df_airlines = spark.read.parquet("/kaggle/input/flight-delay-dataset-20182022/*parquet")

print(f'{df_airlines.count():} flight records loaded')

29193782 flight records loaded


In [19]:
df_airlines.show()

+-------------------+-----------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+-----------------+
|         FlightDate|          Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|D

In [18]:
# carrier delay 12 hours window before prediction time
# carrier delay 14 hours and 15 minutes (-51,300 seconds) to 2 hours and 15 minutes (-8,100 seconds) before scheduled departure time

carrier_window = Window.partitionBy('OP_CARRIER')\
                       .orderBy(unix_timestamp('dep_datetime_scheduled_utc'))\
                       .rangeBetween(-51300, -8100)

df_airlines = df_airlines.withColumn('carrier_delay', round(avg(col('DEP_DEL15')).over(carrier_window),4) )

24/02/19 00:57:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `DEP_DEL15` cannot be resolved. Did you mean one of the following? [`ArrDel15`, `DepDel15`, `Dest`, `AirTime`, `Airline`].;
'Project [FlightDate#0, Airline#1, Origin#2, Dest#3, Cancelled#4, Diverted#5, CRSDepTime#6L, DepTime#7, DepDelayMinutes#8, DepDelay#9, ArrTime#10, ArrDelayMinutes#11, AirTime#12, CRSElapsedTime#13, ActualElapsedTime#14, Distance#15, Year#16L, Quarter#17L, Month#18L, DayofMonth#19L, DayOfWeek#20L, Marketing_Airline_Network#21, Operated_or_Branded_Code_Share_Partners#22, DOT_ID_Marketing_Airline#23L, ... 39 more fields]
+- Relation [FlightDate#0,Airline#1,Origin#2,Dest#3,Cancelled#4,Diverted#5,CRSDepTime#6L,DepTime#7,DepDelayMinutes#8,DepDelay#9,ArrTime#10,ArrDelayMinutes#11,AirTime#12,CRSElapsedTime#13,ActualElapsedTime#14,Distance#15,Year#16L,Quarter#17L,Month#18L,DayofMonth#19L,DayOfWeek#20L,Marketing_Airline_Network#21,Operated_or_Branded_Code_Share_Partners#22,DOT_ID

# EDA